In [1]:
import graphlab

products = graphlab.SFrame('amazon_baby.gl/')
products.head(10)

[INFO] This trial license of GraphLab Create is assigned to chengzh2008@gmail.com and will expire on October 23, 2015. Please contact trial@dato.com for licensing options or to request a free non-commercial license for personal or academic use.

[INFO] Start server at: ipc:///tmp/graphlab_server-39374 - Server binary: /Users/xiaoyazi/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1443844430.log
[INFO] GraphLab Server Version: 1.6.1


name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


In [2]:
# build the word count vector for each review
products['word_count'] = graphlab.text_analytics.count_words(products['review'])
products.head(10)

# Categorical view in the browser
graphlab.canvas.set_target('browser')
# products['name'].show()

products['rating'].show('Categorical')

Canvas is accessible via web browser at the URL: http://localhost:49501/index.html
Opening Canvas in default web browser.


In [3]:
## define what is positive (rating >= 4) and negative (rating <= 2)
# ignore all products whose rating is 3
products = products[products['rating'] != 3]
products['rating'].show('Categorical')

Canvas is accessible via web browser at the URL: http://localhost:49501/index.html
Opening Canvas in default web browser.


In [4]:

# create sentiment column
products['sentiment'] = products['rating'] >= 4
products['sentiment'].show('Categorical')

Canvas is accessible via web browser at the URL: http://localhost:49501/index.html
Opening Canvas in default web browser.


In [5]:

# fine tune the modeling with selected words.
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

# create columns for each row the counts of those selected words.

for word in selected_words:
    products[word] = products['word_count'].apply(lambda word_count_dic: word_count_dic[word] if word in word_count_dic.keys() else 0, dtype=int)
    print 'the sume of this word %s is : %s' % (word, products[word].sum())
products.show()

the sume of this word awesome is : 2002
the sume of this word great is : 42420
the sume of this word fantastic is : 873
the sume of this word amazing is : 1305
the sume of this word love is : 40277
the sume of this word horrible is : 659
the sume of this word bad is : 3197
the sume of this word terrible is : 673
the sume of this word awful is : 345
the sume of this word wow is : 131
the sume of this word hate is : 1057
Canvas is accessible via web browser at the URL: http://localhost:49501/index.html
Opening Canvas in default web browser.


In [23]:
# train the sentiment classifier
train_data, test_data = products.random_split(.8, seed=0)
print "train_data: %s, test_data: %s" % (len(train_data), len(test_data))

test_data['sentiment'].show('Categorical')

train_data: 133448, test_data: 33304
Canvas is updated and available in a tab in the default browser.


In [7]:
sentiment_model = graphlab.logistic_classifier.create(train_data, target='sentiment', features=['word_count'], validation_set=test_data)
print sentiment_model['coefficients'].sort('value', ascending=False)

selected_words_model = graphlab.logistic_classifier.create(train_data, target='sentiment', features=selected_words, validation_set=test_data)
print selected_words_model['coefficients'].sort('value', ascending=False)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 219217
PROGRESS: Number of coefficients    : 219218
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 5        | 0.000002  | 2.312711     | 0.841481          | 0.839989            |
PROGRESS: | 2         | 9        | 3.000000  | 3.595585     | 0.947425          | 0.894877            |
PROGRESS: | 3         | 10       | 3.000000  | 4.019586     | 0.92

In [22]:

print 'evaluation.....'
sentiment_model.evaluate(test_data, metric='roc_curve')
sentiment_model.show(view='Evaluation')


# get the data for a particual product: giraffe and its categorical view
# giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']
# giraffe_reviews['rating'].show(view='Categorical')

# apply the learned model to understand sentiment for Giraffe
# giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')
# print giraffe_reviews.head(30)

# sort the review based on the predicted sentiment and explore
# giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

# print giraffe_reviews[0]['review']



evaluation.....
Canvas is updated and available in a tab in the default browser.


In [21]:
selected_words_model.evaluate(test_data, metric='roc_curve')
selected_words_model.show(view='Evaluation')

Canvas is updated and available in a tab in the default browser.


In [19]:
print " understanding the review..."
# get the data for another product: Baby Trend Diaper Champ
baby_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

baby_champ_reviews['predicted_sentiment'] = selected_words_model.predict(baby_champ_reviews, output_type='probability')
baby_champ_reviews = baby_champ_reviews.sort('rating', ascending=False)
print selected_words_model.predict(baby_champ_reviews[0:1], output_type='probability')

baby_champ_reviews['predicted_sentiment'] = sentiment_model.predict(baby_champ_reviews, output_type='probability')
baby_champ_reviews = baby_champ_reviews.sort('rating', ascending=False)
print sentiment_model.predict(baby_champ_reviews[0:1], output_type='probability')

for word in selected_words:
    print baby_champ_reviews[0][word]


 understanding the review...
[0.796940851290673]
[0.9999606821195993]
0
0
0
0
0
0
0
0
0
0
0
